In [ ]:
#|default_exp app_v2

In [1]:
#| export
from fastai.tabular.all import *

import seaborn as sns

import tqdm

pd.options.display.max_rows = 20
pd.options.display.max_columns = 8

In [2]:
#| export
try: import fastkaggle
except ModuleNotFoundError:
    !pip install -Uq fastkaggle

from fastkaggle import *

#### Downloading Datasets

In [3]:
#| export
comp = 'amp-parkinsons-disease-progression-prediction'
path = setup_comp(comp, install='fastai')

#### Create Dataframes

Training Data

In [4]:
#| export
train_peptides = pd.read_csv(path/'train_peptides.csv')
train_proteins = pd.read_csv(path/'train_proteins.csv')
train_clinical_data = pd.read_csv(path/'train_clinical_data.csv')
supplemental_clinical_data = pd.read_csv(path/'supplemental_clinical_data.csv')

In [12]:
import nbdev
nbdev.export.nb_export('pb_parkinsons_prog.ipynb', 'app_v2')
print("export successful")

export successful


In [14]:
import re

with open('./app_v2/app_v1.py', 'r') as f:
    lines = f.readlines()

with open('./app_v2/app_v1_clean.py', 'w') as f:
    for line in lines:
        if not re.match(r'# %% ../pb_parkinsons_prog\.ipynb \d+', line):
            f.write(line)